In [17]:
import numpy as np
import pandas as pd
import pandas as pd
from pathlib import Path

results_dir = Path("/Users/ryansteele/my_repo2/reports")

# test data
files_test = sorted(results_dir.glob("predictions_test_*.csv"))

print("Found files:", [f.name for f in files_test])

dfs = [pd.read_csv(f) for f in files_test]
df_test_all = pd.concat(dfs, ignore_index=True)
df_test_all = df_test_all.sort_values(["season", "week", "team"]).reset_index(drop=True)

# train data
files_train = sorted(results_dir.glob("predictions_train_*.csv"))

print("Found files:", [f.name for f in files_train])

dfs = [pd.read_csv(f) for f in files_train]
df_train_all = pd.concat(dfs, ignore_index=True)
df_train_all = df_train_all.sort_values(["season", "week", "team"]).reset_index(drop=True)

y_true = df_test_all["y_true"]
baseline_pred_mean = df_train_all["y_true"].mean()

baseline_mae = np.mean(np.abs(y_true - baseline_pred_mean))
baseline_rmse = np.sqrt(np.mean((y_true - baseline_pred_mean)**2))

print("Baseline Mean PD:", baseline_pred_mean)
print("Baseline MAE:", baseline_mae)
print("Baseline RMSE:", baseline_rmse)

Found files: ['predictions_test_2023.csv', 'predictions_test_2024.csv', 'predictions_test_2025.csv']
Found files: ['predictions_train_2023.csv', 'predictions_train_2024.csv', 'predictions_train_2025.csv']
Baseline Mean PD: 2.1911764705882355
Baseline MAE: 10.92361677344205
Baseline RMSE: 14.029095601614015


### Baseline 1 (Above):
Baseline 1 is just the historical mean of point differentials (2023-2025). The 
evaluation metrics use y_true - historical_mean. The baseline of 10.92 is 
conservative and a poor predictor of game outcomes (predicts ~2 for every game).
The XGBoost model outperforms the baseline substantially with a MAE of ~9.7
across test seasons. XGBoost model RMSE is also comfortably better than the 
baseline, with an RMSE ~12.3 across test seasons.

In [19]:
y_true = df_test_all["y_true"]
baseline_pred_vegas = -df_test_all["vegas_spread"]

baseline_vegas_mae = np.abs(y_true - baseline_pred_vegas).mean()
baseline_vegas_rmse = np.sqrt(((y_true - baseline_pred_vegas)**2).mean())

print("Vegas Spread MAE:", baseline_vegas_mae)
print("Vegas Spread RMSE:", baseline_vegas_rmse)

Vegas Spread MAE: 9.40990099009901
Vegas Spread RMSE: 12.055359105738694


### Baseline 2 (Vegas Spreads):

The second baseline used is y_true - vegas_spread. This is used to get a good
comparison between XGBoost model and Vegas models (golden standard). This
project's model is just *slightly* worse than the Vegas model—difference between
Vegas and XGBoost models: MAE: ~0.3; RMSE: ~0.25. This project's model is
incredibly close to the golden standard, which is impressive considering
this project doesn't have historical injury data and weather data (strong
predictors).